In [1]:
#入室管理システム
#常時カメラ撮影→顔認識で特定→姿勢検出より最適な位置に姿勢が固定されたときに撮影を行う。撮影した画像と個人名、日付を紐付ける


In [1]:
import face_recognition
import cv2
import numpy as np
import tensorflow as tf
import time


known_face_names=[]
known_face_encodings=[]
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

f=open("HumanName/NameList.txt", "r")
line=f.readline()
while line:
    line=line.strip()
    print(line)
    face_image = face_recognition.load_image_file(line)
    face_encoding = face_recognition.face_encodings(face_image)[0]
    known_face_names.append(line)
    known_face_encodings.append(face_encoding)
    line = f.readline()

print(len(known_face_names))

#Body_segmentation前処理
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
bodypix_model = load_model(download_model(BodyPixModelPaths.RESNET50_FLOAT_STRIDE_16))

#Posenet
from posenet.posenet_factory import load_model
from posenet.utils import draw_skel_and_kp

model = 'mobilenet'  # mobilenet resnet50
stride = 16  # 8, 16, 32 (max 16 for mobilenet, min 16 for resnet50)
quant_bytes = 4  # float
multiplier = 0.5  # only for mobilenet
posenet = load_model(model, stride, quant_bytes, multiplier)


#顔認証前処理
cap = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = cap.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)
    
    # 画像の取得と顔の検出

    image = frame

    image_array = tf.keras.preprocessing.image.img_to_array(image)
    result = bodypix_model.predict_single(image_array)
    mask = result.get_mask(threshold=0.75)

    colored_mask = result.get_colored_part_mask(mask)
    
    colored_mask=np.array(colored_mask,dtype=np.uint8)
    
    
    # BGRでの色抽出
    bgrLower = np.array([135, 240, 87])     # 抽出する色の下限(BGR)
    bgrUpper = np.array([175, 245, 91])   # 抽出する色の上限(BGR)
    img_mask = cv2.inRange(colored_mask, bgrLower, bgrUpper) # BGRからマスクを作成
    result = cv2.bitwise_and(image, image, mask=img_mask) # 元画像とマスクを合成
    
    if not ret:
        raise IOError("webcam failure")

    pose_scores, keypoint_scores, keypoint_coords = posenet.estimate_multiple_poses(frame)

    overlay_image = draw_skel_and_kp(
        frame, pose_scores, keypoint_scores, keypoint_coords,
        min_pose_score=0.15, min_part_score=0.1)

    cv2.imshow('posenet', overlay_image)

    cv2.imshow("nomal",frame)
    cv2.imshow("color",result)
    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release handle to the webcam
cap.release()
cv2.destroyAllWindows()

HumanName/biden.jpg
HumanName/obama.jpg
2
Loading MobileNet model
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
We use the signature key serving_default It should be in the keys list:
signature key: serving_default
model outputs: {'MobilenetV1/offset_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/offset_2/BiasAdd:0' shape=(1, None, None, 34) dtype=float32>, 'MobilenetV1/displacement_bwd_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/displacement_bwd_2/BiasAdd:0' shape=(1, None, None, 32) dtype=float32>, 'MobilenetV1/heatmap_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/heatmap_2/BiasAdd:0' shape=(1, None, None, 17) dtype=float32>, 'MobilenetV1/displacement_fwd_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/displacement_fwd_2/BiasAdd:0' shape=(1, None, None, 32) dtype=float32>}


In [7]:
import cv2
import tensorflow as tf
from tf_bodypix.api import download_model, load_model, BodyPixModelPaths
import numpy as np

bodypix_model = load_model(download_model(
    BodyPixModelPaths.RESNET50_FLOAT_STRIDE_16
))


# カメラ映像取得
cap = cv2.VideoCapture(0)


# 変換処理ループ
while True:
    
    # 初期フレームの読込
    end_flag, c_frame = cap.read()
    # 画像の取得と顔の検出

    image = c_frame

    image_array = tf.keras.preprocessing.image.img_to_array(image)
    result = bodypix_model.predict_single(image_array)
    mask = result.get_mask(threshold=0.75)

    colored_mask = result.get_colored_part_mask(mask)
    
    colored_mask=np.array(colored_mask,dtype=np.uint8)
    
    
    # BGRでの色抽出
    bgrLower = np.array([135, 240, 87])     # 抽出する色の下限(BGR)
    bgrUpper = np.array([175, 245, 91])   # 抽出する色の上限(BGR)
    img_mask = cv2.inRange(colored_mask, bgrLower, bgrUpper) # BGRからマスクを作成
    result = cv2.bitwise_and(image, image, mask=img_mask) # 元画像とマスクを合成


    # フレーム表示
    cv2.imshow("nomal",c_frame)
    cv2.imshow("color",result)
    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# 終了処理
cap.release()
cv2.destroyAllWindows()

In [5]:
#posenetの姿勢制御
import tensorflow as tf
import cv2

from posenet.posenet_factory import load_model
from posenet.utils import draw_skel_and_kp

model = 'mobilenet'  # mobilenet resnet50
stride = 16  # 8, 16, 32 (max 16 for mobilenet, min 16 for resnet50)
quant_bytes = 4  # float
multiplier = 0.5  # only for mobilenet

posenet = load_model(model, stride, quant_bytes, multiplier)

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 360)


while True:
    res, img = cap.read()
    if not res:
        raise IOError("webcam failure")

    pose_scores, keypoint_scores, keypoint_coords = posenet.estimate_multiple_poses(img)

    overlay_image = draw_skel_and_kp(
        img, pose_scores, keypoint_scores, keypoint_coords,
        min_pose_score=0.15, min_part_score=0.1)

    cv2.imshow('posenet', overlay_image)
    frame_count += 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Loading MobileNet model
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
We use the signature key serving_default It should be in the keys list:
signature key: serving_default
model outputs: {'MobilenetV1/heatmap_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/heatmap_2/BiasAdd:0' shape=(1, None, None, 17) dtype=float32>, 'MobilenetV1/offset_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/offset_2/BiasAdd:0' shape=(1, None, None, 34) dtype=float32>, 'MobilenetV1/displacement_bwd_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/displacement_bwd_2/BiasAdd:0' shape=(1, None, None, 32) dtype=float32>, 'MobilenetV1/displacement_fwd_2/BiasAdd:0': <tf.Tensor 'MobilenetV1/displacement_fwd_2/BiasAdd:0' shape=(1, None, None, 32) dtype=float32>}


In [4]:
#mediapipeで姿勢制御をするプログラム。重いため第二候補

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose



# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
      while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = pose.process(image)

        # Draw the pose annotation on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('MediaPipe Pose', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()